In [1]:
!pip install langchain
!pip install langchain_community
!pip install PyPDF

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 735.7 kB/s eta 0:00:00 kB/s eta 0:00:01:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [30]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install -qU langchain-text-splitters


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
!pip install llama-cpp-python

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 3.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.81-cp310-cp310-linux_x86_64.whl size=2779307 sha256=8476dd04a78f4351197082581c924a39fa7dbb375f616ecb62d899a30893da24
  Stored in directory: /home/nayra/.cache/pip/wheels/7c/4a/7f/592e73429da89e4086bbd65cda7197ab53c403c17d386894cd
Successfully built llama-cpp-python


In [37]:
!pip install langchain_chroma

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 KB 924.3 kB/s eta 0:00:00MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 KB 1.4 MB/s eta 0:00:001.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 KB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:01:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 2.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 KB 3.5 MB/s eta 0:00:003.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0

In [53]:
from langchain_community.document_loaders import PyPDFLoader

def loader_pdf(path):
    loader = PyPDFLoader(path)
    return loader.load()

text = loader_pdf('/home/nayra/Downloads/JavaBasico.pdf')

In [54]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def processing_text(text):
    
    texto = []
    for i in range(len(text)):
        texto.append(" "+text[i].page_content)
            
    text_splitter = RecursiveCharacterTextSplitter(
    separators=[
            "\n\n",
        ],
        # Existing args
    )
    
    texts = text_splitter.split_documents(text)

    return texts

texto_novo = processing_text(text)

[Document(page_content='Volume único\nJava Básico e \nOrientação a ObjetoClayton Escouper das Chagas\nCássia Blondet Baruque\nLúcia Blondet Baruque', metadata={'source': '/home/nayra/Downloads/JavaBasico.pdf', 'page': 0}),
 Document(page_content='Clayton Escouper das Chagas\nCássia Blondet BaruqueLúcia Blondet BaruqueVolume únicoJava Básico e \nOrientação a Objeto\nApoio:', metadata={'source': '/home/nayra/Downloads/JavaBasico.pdf', 'page': 2}),
 Document(page_content='2010/1Material Didático\nORGANIZAÇÃO\nCássia Blondet Baruque\nELABORAÇÃO DE CONTEÚDO\nClayton Escouper das Chagas\nCássia Blondet BaruqueLúcia Blondet BaruqueFundação Cecierj / Extensão\nRua Visconde de Niterói, 1364 – Mangueira – Rio de Janeiro, RJ – CEP 20943-001\nTel.: (21) 2334-1569 Fax: (21) 2568-0725\nPresidente\nMasako Oya Masuda\nVice-presidente\nMirian Crapez\nCoordenadoras da Área de Governança: Gestão, Auditoria \ne Tecnologia da Informação\nLúcia Blondet Baruque\nCássia Blondet Baruque\nEDITORA\nTereza Queiro

In [57]:
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings

def vetoring_text(text):
    embedder = SpacyEmbeddings(model_name="pt_core_news_md")
    
    docs = []
    for i in range(len(text)):
        docs.append(text[i].page_content)
    
    embeddings = embedder.embed_documents(docs)
    return embeddings

embeddings = vetoring_text(texto_novo)

[[0.40350618958473206, 0.06341050565242767, -0.13883936405181885, 0.02669616788625717, 0.011409049853682518, -0.40135854482650757, -0.5663946866989136, -0.26252755522727966, -1.064342975616455, -0.401934951543808, 0.10325732827186584, -0.24139690399169922, 0.9423966407775879, 0.4897095859050751, 0.08615285158157349, -0.3184930384159088, 0.7539797425270081, 0.253339946269989, 0.423743337392807, 0.6683909893035889, 0.011582477949559689, 0.14922453463077545, 0.09424944967031479, -0.5780938267707825, -0.49666526913642883, 1.1358652114868164, 0.01648286171257496, -0.15383584797382355, 0.40304192900657654, -0.03255148604512215, -0.040394753217697144, 0.2291690558195114, 0.23922483623027802, -0.8207429051399231, 0.9518500566482544, -0.5020319819450378, 0.0065733902156353, 0.02262808009982109, 0.058169540017843246, -0.2024666666984558, 0.18159453570842743, 0.037292394787073135, 0.3770514726638794, -0.4942595064640045, -0.061219509690999985, -0.007279535289853811, -0.01193428598344326, 0.048961

In [58]:
from langchain_chroma import Chroma
import chromadb

client = chromadb.PersistentClient(path='/home/nayra/Documentos/chroma')
collection = client.create_collection(name="oiiiiiijk")